# Using tensorboard
This notebook demonstrates how to log data to tensorboard during training and how to use tensorboard to visualize the model. Check out chapter 4 of the book to learn more on how to use tensorboard.

## The model
We're using the iris flower classification model from previous chapters. 
The model has 4 input neurons and 3 output neurons. We'll be using a softmax activation on the output layer to create a classification model.

In [31]:
from cntk import default_options, input_variable
from cntk.layers import Dense, Sequential
from cntk.ops import log_softmax, relu, sigmoid

model = Sequential([
    Dense(4, activation=relu),
    Dense(3, activation=log_softmax)
])

features = input_variable(4)
labels = input_variable(3)

z = model(features)

## Data preprocessing
We're going to use the original CSV file for the iris dataset.
It contains four columns with features. The final column is the label (the species of the sample).

The labels are stored as string and we need to have a binary encoded variant.
So we'll use a Binarizer from scikit-learn to encode the labels in the proper format.

In [32]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelBinarizer

df_source = pd.read_csv('iris.csv', 
    names=['sepal_length', 'sepal_width','petal_length','petal_width', 'species'], 
    index_col=False)

label_encoder = LabelBinarizer()

X = df_source.iloc[:, :4].values
y = df_source.iloc[:, -1:].values

y = label_encoder.fit_transform(y)

X = X.astype(np.float32)
y = y.astype(np.float32)

## Split the dataset into a training and test set
Before we start training, we'll split the dataset into a training and test set using the `train_test_split` function from scikit-learn. This gives us a small set of samples to later validate the model on.

In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, stratify=y)

## Train the model
Now that we have a dataset, let's train the model with it.
For this we define a loss and a learner. We can then use the loss to train the model.

In [34]:
import cntk

@cntk.Function
def criterion_factory(output, target):
    loss = cntk.losses.cross_entropy_with_softmax(output, target)
    metric = cntk.losses.fmeasure(output, target, beta=1)
    
    return loss, metric

We've included a tensorboard progress writer in the trainer callbacks so that metrics are logged in the native tensorboard format. You can view the data by running the command: `tensorboard --logdir logs` in the directory that this notebook is stored.

In [35]:
import time
from cntk import DataUnit
from cntk.train.training_session import CrossValidationConfig
from cntk.losses import cross_entropy_with_softmax
from cntk.learners import sgd 
from cntk.logging import ProgressPrinter, TensorBoardProgressWriter

loss = criterion_factory(z, labels)
learner = sgd(z.parameters, 0.1)

train_callbacks = [
    ProgressPrinter(freq=0, log_to_file='test_log.txt'),
    TensorBoardProgressWriter(log_dir='logs/{}'.format(time.time()), freq=1, model=z)
]

train_summary = loss.train((X_train,y_train), 
                           parameter_learners=[learner], 
                           callbacks=train_callbacks, 
                           minibatch_size=16, max_epochs=15)

Redirecting log to file test_log.txt


Close the tensorboard connection after you're done. This ensures that all logging data is flushed to disk. Otherwise you may miss data  in tensorboard.

In [36]:
train_callbacks[1].close()